# Setup

In [1]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()

# We're testing the code directly, not the package installed in the environment
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('.'), '..')))

print(sys.path)

['/Users/maykcaldas/Documents/WhiteLab/BO-LIFT', '/Users/maykcaldas/miniconda3/envs/bolift/lib/python310.zip', '/Users/maykcaldas/miniconda3/envs/bolift/lib/python3.10', '/Users/maykcaldas/miniconda3/envs/bolift/lib/python3.10/lib-dynload', '', '/Users/maykcaldas/miniconda3/envs/bolift/lib/python3.10/site-packages']


# Test LLM

In [2]:
import bolift
from bolift import llm_model
 

/Users/maykcaldas/miniconda3/envs/bolift/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
system_message = "Answer the number ONLY. For example, if the question is 'What is 1 + 1?', answer '2'."

query = "The value of 4 + 4 is"

print(query)
llm = llm_model.get_llm(model_name="claude-3-5-sonnet-20240620", stop=["\n\n"], use_logprobs=True)
result, token = llm.predict(query, system_message=system_message)
f"The answer is {result} and it used {token} tokens"

The value of 4 + 4 is


/Users/maykcaldas/Documents/WhiteLab/BO-LIFT/bolift/llm_model.py:357: UserWarning: Logprobs not supported for Anthropic models. Ignoring `use_logprobs` parameter.
  warnings.warn("Logprobs not supported for Anthropic models. Ignoring `use_logprobs` parameter.")


'The answer is [GaussDist(8.0, None)] and it used 52 tokens'

In [4]:
system_message = "Answer the number ONLY. For example, if the question is 'What is 1 + 1?', answer '2'."

query = "What is the solubility of octanal?"

print(query)
for lgp in [True, False]:
  llm = llm_model.get_llm(model_name="gpt-4o", stop=["\n\n"], use_logprobs=lgp)
  result, token = llm.predict(query, system_message=system_message)
  print(f"With use_logprobs=={lgp} \n\tThe answer is {result} and it used {token} tokens")

What is the solubility of octanal?
With use_logprobs==True 
	The answer is [DiscreteDist([0.03  0.015 0.4   0.5   0.01 ], [0.2723433  0.21210117 0.16518456 0.2723433  0.07802768])] and it used 61 tokens
With use_logprobs==False 
	The answer is [DiscreteDist([0.05  0.005 0.03  0.2   0.4  ], [0.2 0.2 0.2 0.2 0.2])] and it used 61 tokens


In [5]:
system_message = "Answer the number ONLY. For example, if the question is 'What is 1 + 1?', answer '2'."

query = "What is the solubility of octanal?"

print(query)
for lgp in [True, False]:
  llm = llm_model.get_llm(model_name="gpt-3.5-turbo-instruct", stop=["\n\n"], use_logprobs=lgp)
  result, token = llm.predict(query, system_message=system_message)
  print(f"With use_logprobs=={lgp} \n\tThe answer is {result} and it used {token} tokens")

What is the solubility of octanal?


/Users/maykcaldas/miniconda3/envs/bolift/lib/python3.10/site-packages/bolift/llm_model.py:228: RuntimeWarning: invalid value encountered in divide
  probs = probs / np.sum(probs)


With use_logprobs==True 
	The answer is [GaussDist(nan, None)] and it used 131 tokens


Error in OpenAICallbackHandler.on_llm_end callback: TypeError("unsupported operand type(s) for /: 'NoneType' and 'int'")


With use_logprobs==False 
	The answer is [DiscreteDist([], [])] and it used 0 tokens


# Test AskTell

In [4]:
import bolift
import openai

In [7]:
from bolift import AskTellFewShotTopk

system_message = "Answer the number ONLY. For example, if the question is 'What is 1 + 1?', answer '2'."

query = "8"

asktell = AskTellFewShotTopk(
  model="gpt-4o", 
  use_logprobs=False,
  x_formatter=lambda x: f"y = 2*{x}",
  y_formatter=lambda y: y,
  )

asktell.tell(1, 2)
asktell.tell(2, 4)
asktell.tell(4, 8)
asktell.tell(16, 32)

asktell.predict(query, system_message=system_message) 
# The llm is returning GaussDis(8, None). 12 is the population's std np.std([2, 4, 8, 32]) ~= 12.03

/Users/maykcaldas/Documents/WhiteLab/BO-LIFT/bolift/llm_model.py:159: UserWarning: WARNING! logprobs is not default parameter.
                logprobs was transferred to model_kwargs.
                Please confirm that logprobs is what you intended.
  self.llm = self.create_llm()
/Users/maykcaldas/Documents/WhiteLab/BO-LIFT/bolift/llm_model.py:159: UserWarning: WARNING! top_logprobs is not default parameter.
                top_logprobs was transferred to model_kwargs.
                Please confirm that top_logprobs is what you intended.
  self.llm = self.create_llm()
/Users/maykcaldas/Documents/WhiteLab/BO-LIFT/bolift/llm_model.py:159: UserWarning: WARNING! logprobs is not default parameter.
                logprobs was transferred to model_kwargs.
                Please confirm that logprobs is what you intended.
  self.llm = self.create_llm()
/Users/maykcaldas/Documents/WhiteLab/BO-LIFT/bolift/llm_model.py:159: UserWarning: WARNING! top_logprobs is not default parameter.
        

GaussDist(16.0, 12.031209415515965)

In [27]:
from bolift import AskTellNearestNeighbor

system_message = "Answer the number ONLY. For example, if the question is 'What is 1 + 1?', answer '2'."

query = "8"

asktell = AskTellNearestNeighbor(
  knn=5,
  x_formatter=lambda x: f"y = 2*{x}",
  y_formatter=lambda y: str(y),
  )

asktell.tell(1, 2)
asktell.tell(2, 4)
asktell.tell(4, 8)
asktell.tell(9, 18)
asktell.tell(16, 32)
asktell.tell(32, 64)
asktell.tell(64, 128)

asktell.predict(query)

GaussDist(12.8, 11.070682002478437)

In [39]:
from bolift import AskTellRidgeKernelRegression

system_message = "Answer the number ONLY. For example, if the question is 'What is 1 + 1?', answer '2'."

query = "8"

asktell = AskTellRidgeKernelRegression(
  x_formatter=lambda x: f"y = 2*{x}",
  y_formatter=lambda y: str(y),
  )

asktell.tell(1, 2, train=False)
asktell.tell(2, 4, train=False)
asktell.tell(4, 8, train=False)
asktell.tell(9, 18, train=False)
asktell.tell(16, 32, train=False)
asktell.tell(32, 64, train=True) #Default is True

asktell.predict(query)

Cached embeddings not found. Creating new cache table.


GaussDist(25.342013093575382, 0.0)

In [1]:
import os
import sys
import json
from dotenv import load_dotenv
load_dotenv()
from bolift import AskTellFinetuning

system_message = "Answer the number ONLY. For example, if the question is 'What is 1 + 1?', answer '2'."

query = "8"

asktell = AskTellFinetuning(
  model="gpt-4o-mini-2024-07-18", 
  x_formatter=lambda x: f"y = 2*{x}",
  y_formatter=lambda y: y,
  )

prompts = ["2", "4", "8"]*4
completions = ["4", "8", "16"]*4

for p, c in zip(prompts, completions):
  asktell.tell(p, c)
asktell.tell("1", "2", train=True)

/Users/maykcaldas/miniconda3/envs/bolift/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/maykcaldas/miniconda3/envs/bolift/lib/python3.10/site-packages/bolift/llm_model.py:130: UserWarning: Model gpt-4o-mini-2024-07-18 not explicitly supported. Please choose from ['davinci-002', 'gpt-3.5-turbo-instruct', 'gpt-4', 'gpt-3.5-turbo', 'gpt-4-turbo-preview', 'gpt-3.5-turbo-0125', 'gpt-4-0125-preview', 'gpt-4o', 'gpt-4o-mini', 'meta-llama/Llama-2-7b-chat-hf', 'meta-llama/Llama-2-13b-chat-hf', 'meta-llama/Llama-2-70b-chat-hf', 'mistralai/Mistral-7B-Instruct-v0.1', 'mistralai/Mixtral-8x7B-Instruct-v0.1']

We will try to use this model as a ChatOpenAI model.
  warnings.warn(f"Model {model_name} not explicitly supported. Please choose from {openai_models + chatopenai_models + anyscale_models}\n\nWe will try

RateLimitError: Error code: 429 - {'error': {'message': "This fine-tune request has been rate-limited. Your organization has reached the maximum of 10 fine-tuning requests per day for the model 'gpt-4o-mini-2024-07-18'.", 'type': 'invalid_request_error', 'param': None, 'code': 'daily_rate_limit_exceeded'}}

In [ ]:
asktell

In [2]:
from openai import OpenAI

client = OpenAI()

s = client.fine_tuning.jobs.retrieve("ftjob-8A7GOI5oP3TY409xBcBNKGVh")

s.status



'succeeded'

In [14]:
# asktell2 = AskTellFinetuning(id="ftjob-8A7GOI5oP3TY409xBcBNKGVh")

# asktell2.llm.predict("8", system_message=system_message)
json.dumps(dict(s).__str__(), indent=4)

'"{\'id\': \'ftjob-8A7GOI5oP3TY409xBcBNKGVh\', \'created_at\': 1723135320, \'error\': Error(code=None, message=None, param=None), \'fine_tuned_model\': \'ft:gpt-4o-mini-2024-07-18:white-research-laboratory::9u0eIwtK\', \'finished_at\': 1723135888, \'hyperparameters\': Hyperparameters(n_epochs=6, batch_size=1, learning_rate_multiplier=1.8), \'model\': \'gpt-4o-mini-2024-07-18\', \'object\': \'fine_tuning.job\', \'organization_id\': \'org-zVzDC0J6UhWoGf9pmQAfuLud\', \'result_files\': [\'file-0nx2hSLUYwOuz3N3gjiEcj4p\'], \'seed\': 1988874339, \'status\': \'succeeded\', \'trained_tokens\': 1350, \'training_file\': \'file-rREEijVKOlTLtaOH9JD4e8GY\', \'validation_file\': None, \'estimated_finish\': None, \'integrations\': [], \'user_provided_suffix\': None}"'